In [2]:
%matplotlib inline
import os
import time
import myblack
import numpy as np
from astropy import wcs
from astropy import constants as cons
from astropy import units as u
from astropy.table import Column
from astropy.io import ascii, fits
from matplotlib import pyplot as plt
import matplotlib as mpl
from astropy.table import Column, Table
from lmfit import minimize, Parameters, Model

mpl.rc("font", family="serif", size=15)
mpl.rc("axes", linewidth =  1 )
mpl.rc("lines", linewidth = 1 )
mpl.rc("xtick.major", pad = 8, size = 8, width = 1)
mpl.rc("ytick.major", pad = 8, size = 8, width = 1)
mpl.rc("xtick.minor", size = 4, width = 1 )
mpl.rc("ytick.minor", size = 4, width = 1 )

In [3]:
sourList = ascii.read('./sourList.txt')

In [4]:
# Initiate the parameters
iMass  = 1000*u.Msun # in cm^-2
iTdust = 22.0 # in K.
ibeta  = 2.0 # 
betaVary = False # If True, Beta will be fitted.
                 # If False, beta will be fixed.
betaMin = 1.0
betaMax = 3.0

# Constants

h = cons.h.cgs.value   # Planck constant in CGS unit
k = cons.k_B.cgs.value # Boltzmann constant in CGS unit
c = cons.c.cgs.value # speed of light in CGS unit
mH = cons.m_n.cgs.value # mass of an neutron
muh2 = 2.8 # mean molecular weight adopted from Kauffmann et al. (2008)
rGD = 100.0 # gas-to-dust mass ratio
nu0 = 599.584916E9 # Reference frequency in Hz.
kappa0 = 5.0 # Dust emissivity at reference frequency

def greybody(nu, mass=iMass.cgs.value, Tdust=iTdust, beta=ibeta):
    
    blackBody = 2*h*nu**3/c**2/(np.exp(h*nu/k/Tdust)-1)
    tau = mass*kappa0*(nu/nu0)**beta/d**2/rGD
    return blackBody*tau

gbMod = Model(greybody)

gbMod.set_param_hint('beta', min = betaMin, max = betaMax, 
        vary = betaVary)
gbMod.set_param_hint('Tdust',  max = 80.0)

pars = gbMod.make_params(mass = iMass.cgs.value, Tdust = iTdust)

In [5]:
pfmt = '%6i %18s %6.2f%% finished.'

Tdust_col = []
errTdust_col = []
Menv_col = []
errMenv_col = []

for isour in range(len(sourList)):
    sName = sourList['Name'][isour]
    dis   = sourList['dis'][isour]*u.kpc
    d   = dis.cgs.value # distance in cm
    fluxData = np.array([sourList['S160'][isour], sourList['S250'][isour], 
                         sourList['S350'][isour], sourList['S500'][isour],
                         sourList['S870'][isour]])*u.Jy
    
    fluxErr = fluxData*0.2

    wavelengths = np.array([160,250,350, 500, 870]) * u.um
    frequencies = wavelengths.to(u.Hz, u.spectral()).value.copy()
    
    sedResult = gbMod.fit(fluxData.cgs.value, nu = frequencies)
    
    T    = sedResult.params['Tdust'].value
    errT = sedResult.params['Tdust'].stderr
    B    = sedResult.params['beta'].value
    errB = sedResult.params['beta'].stderr
    M    = sedResult.params['mass'].value*u.g
    errM = sedResult.params['mass'].stderr*u.g
    
    Tdust_col.append(T)
    errTdust_col.append(errT)
    Menv_col.append(M.to(u.Msun).value)
    errMenv_col.append(errM.to(u.Msun).value)
    
    # plotting the SED


    x = np.array(np.logspace(1,3.5, 300))*u.um
    xFreq = x.to(u.Hz, u.spectral()).value.copy()
    y = greybody(xFreq,Tdust = sedResult.params['Tdust'].value,
                 mass= sedResult.params['mass'].value)*u.g/u.s/u.s

    fig = plt.figure(figsize = (5,5))
    ax  = fig.add_axes([0.1,0.1, 0.8, 0.6])
    ax.plot(x,y.to(u.Jy).value, '-k')
    ax.scatter(wavelengths, fluxData, color = 'black')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(8, 5000)
    ax.set_ylim(np.e**(np.log(np.min(fluxData.value))-
                     0.2*(np.log(np.max(fluxData.value))
                          -np.log(np.min(fluxData.value)))),
                np.e**(np.log(np.max(fluxData.value))+
                     0.5*(np.log(np.max(fluxData.value))
                          -np.log(np.min(fluxData.value)))))

    xmin,xmax = ax.get_xlim()
    ymin,ymax = ax.get_ylim()
    
    ax.text(10**(np.log10(xmin)+(np.log(xmax)-np.log(xmin))*0.03), 
            10**(np.log10(ymax)-(np.log(ymax)-np.log(ymin))*0.04), 
            sName, horizontalalignment='left')
    
    mLabel = ("${ =\ "+'%d'%M.to(u.Msun).value+
              "\pm"+'%d'%errM.to(u.Msun).value+"\ M_{\odot}}$")
    ax.text(10**(np.log10(xmin)+(np.log(xmax)-np.log(xmin))*0.03), 
            10**(np.log10(ymax)-(np.log(ymax)-np.log(ymin))*0.075), 
            "$M_{env}$", horizontalalignment='left')
    ax.text(10**(np.log10(xmin)+(np.log(xmax)-np.log(xmin))*0.075), 
            10**(np.log10(ymax)-(np.log(ymax)-np.log(ymin))*0.075), 
            mLabel, horizontalalignment='left')
    temLabel = ("${=\ "+'%.1f'%T+
              "\pm"+'%.1f'%errT+"\ K}$")
    ax.text(10**(np.log10(xmin)+(np.log(xmax)-np.log(xmin))*0.03), 
            10**(np.log10(ymax)-(np.log(ymax)-np.log(ymin))*0.11),
            "$T_{dust}$", horizontalalignment='left')
    ax.text(10**(np.log10(xmin)+(np.log(xmax)-np.log(xmin))*0.075), 
            10**(np.log10(ymax)-(np.log(ymax)-np.log(ymin))*0.11),
            temLabel, horizontalalignment='left')
    
    modelFlux = greybody(frequencies, Tdust = sedResult.params['Tdust'].value,
                         mass= sedResult.params['mass'].value)*u.g/u.s/u.s
    chi2 = (np.sum(((modelFlux.to(u.Jy)-
                 fluxData)/fluxErr)**2)/(len(fluxData)-1))
    chi2Label = ("${=\ "+'%.1f'%chi2+"}$")
    ax.text(10**(np.log10(xmin)+(np.log(xmax)-np.log(xmin))*0.03), 
            10**(np.log10(ymax)-(np.log(ymax)-np.log(ymin))*0.145), 
            "$\chi^2$" , horizontalalignment='left')
    ax.text(10**(np.log10(xmin)+(np.log(xmax)-np.log(xmin))*0.075), 
            10**(np.log10(ymax)-(np.log(ymax)-np.log(ymin))*0.145), 
            chi2Label, horizontalalignment='left')
    ax.set_xlabel("Wavelength ($\mu m$)")
    ax.set_ylabel("Flux (Jy)")
    fig.savefig('./figDir/'+sName+'_singleSED.pdf' ,
                dpi = 300, bbox_inches='tight')
    
    fig.clf()
    print(pfmt %(isour+1, sName, ((isour+1.0) / len(sourList)*100)))
    
# writing out the fitting results

Tdust_col = Column(Tdust_col, name = 'Tdust')
Menv_col  = Column(Menv_col, name = 'Mass_env')
errTdust_col = Column(errTdust_col, name = 'errTdust')
errMenv_col  = Column(errMenv_col, name = 'errMass_env')

sourList.add_columns([Tdust_col, errTdust_col, 
                      Menv_col, errMenv_col])

sourList.write('./sourList._with_sedResults.txt',
               format = 'ascii.ipac')

     1   G045.4212+0.0838   4.00% finished.
     2   G045.5356+0.1411   8.00% finished.
     3   G048.8905-0.2649  12.00% finished.
     4   G049.0548-0.3340  16.00% finished.
     5   G049.2527-0.4106  20.00% finished.
     6   G049.3228-0.3461  24.00% finished.
     7   G049.3538-0.3538  28.00% finished.
     8   G049.4011-0.2263  32.00% finished.
     9   G049.4134-0.3537  36.00% finished.
    10   G049.4739-0.2957  40.00% finished.
    11   G049.4909-0.2857  44.00% finished.
    12   G049.5300-0.3478  48.00% finished.
    13   G305.0943+0.2510  52.00% finished.
    14   G305.0955+0.0877  56.00% finished.
    15   G305.1543+0.0477  60.00% finished.
    16   G305.1721+0.0079  64.00% finished.
    17   G305.2350-0.0231  68.00% finished.
    18   G305.2581+0.3275  72.00% finished.
    19   G305.2719-0.0309  76.00% finished.
    20   G305.3187+0.3130  80.00% finished.


/Users/yuan/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


    21   G305.3834+0.2565  84.00% finished.
    22   G305.4126+0.2061  88.00% finished.
    23   G305.5476-0.0559  92.00% finished.
    24   G305.5890+0.4609  96.00% finished.
    25   G307.6099-0.2937 100.00% finished.
